In [1]:
from pathlib import Path
import shutil
import random
import os
from sklearn.model_selection import train_test_split

In [2]:
Path('./VOC/labels').mkdir(parents=True, exist_ok=True)

In [3]:
!unzip -qq "/content/drive/MyDrive/KDT/분석과제/data/Signpost.zip"

In [4]:
# 어노테이션을 YOLO 형식으로 변환
!git clone https://github.com/ssaru/convert2Yolo.git

Cloning into 'convert2Yolo'...
remote: Enumerating objects: 215, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 215 (delta 38), reused 35 (delta 35), pack-reused 172 (from 1)
Receiving objects: 100% (215/215), 994.67 KiB | 3.87 MiB/s, done.
Resolving deltas: 100% (95/95), done.


In [5]:
!pip install -r /content/convert2Yolo/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.1/39.1 MB 21.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 14.0 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [6]:
!python /content/convert2Yolo/example.py \
    --datasets VOC \
    --img_path /content/images \
    --label /content/annotations \
    --convert_output_path /content//VOC/labels \
    --img_type '.png' \
    --manifest_path /content \
    --cls_list_file /content/voc_s.names


VOC Parsing:   |████████████████████████████████████████| 100.0% (877/877)  Complete


YOLO Generating:|████████████████████████████████████████| 100.0% (877/877)  Complete


YOLO Saving:   |████████████████████████████████████████| 100.0% (877/877)  Complete



In [7]:
src = Path("images")          # 원본 images 폴더
dst = Path("VOC") / "images"  # 목적지 VOC/images 폴더

In [8]:
Path('VOC/images').mkdir(parents=True, exist_ok=True)
for p in src.iterdir():
    if p.is_file():
        shutil.copy2(p, dst / p.name)

In [9]:
image_dir = Path('VOC/images')
label_dir = Path('VOC/labels')

image_files = sorted([p for p in image_dir.iterdir() if p.suffix == '.png'])
label_files = sorted([p for p in label_dir.iterdir() if p.suffix == '.txt'])

print(f"Found {len(image_files)} image files.")
print(f"Found {len(label_files)} label files.")

Found 877 image files.
Found 877 label files.


In [10]:
mapped_files = []
for img_path in image_files:
    img_name = img_path.stem  # Get filename without extension
    label_path = label_dir / f"{img_name}.txt"
    if label_path.exists():
        mapped_files.append((img_path, label_path))

print(f"Found {len(mapped_files)} image-label pairs.")


Found 877 image-label pairs.


In [11]:
train_files, test_files = train_test_split(mapped_files, test_size=0.2, random_state=42)

print(f"Number of training files: {len(train_files)}")
print(f"Number of testing files: {len(test_files)}")

Number of training files: 701
Number of testing files: 176


In [12]:
Path('VOC/train/images').mkdir(parents=True, exist_ok=True)
Path('VOC/train/labels').mkdir(parents=True, exist_ok=True)
Path('VOC/test/images').mkdir(parents=True, exist_ok=True)
Path('VOC/test/labels').mkdir(parents=True, exist_ok=True)

print("Created directories: VOC/train/images, VOC/train/labels, VOC/test/images, VOC/test/labels")

Created directories: VOC/train/images, VOC/train/labels, VOC/test/images, VOC/test/labels


In [13]:
for img_path, label_path in train_files:
    shutil.copy2(img_path, Path('VOC/train/images') / img_path.name)
    shutil.copy2(label_path, Path('VOC/train/labels') / label_path.name)

print(f"Copied {len(train_files)} training image-label pairs to VOC/train.")

Copied 701 training image-label pairs to VOC/train.


In [14]:
for img_path, label_path in test_files:
    shutil.copy2(img_path, Path('VOC/test/images') / img_path.name)
    shutil.copy2(label_path, Path('VOC/test/labels') / label_path.name)

print(f"Copied {len(test_files)} testing image-label pairs to VOC/test.")

Copied 176 testing image-label pairs to VOC/test.


# YOLO 학습

In [15]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 27.1 MB/s eta 0:00:00


In [16]:
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [17]:
model = YOLO('yolov8s.pt')

In [18]:
import yaml

# Read class names from voc.names
with open('voc_s.names', 'r') as f:
    class_names = f.read().splitlines()

num_classes = len(class_names)

# Define the YAML content
data_yaml = {
    'path': '/content/VOC', # Adjust if your VOC directory is located elsewhere relative to this file
    'train': 'train/images',
    'val': 'test/images',
    'names': class_names,
    'nc': num_classes
}

# Write the YAML content to a file
with open('custom_s_voc.yaml', 'w') as f:
    yaml.dump(data_yaml, f, default_flow_style=False)

print("custom_s_voc.yaml file created successfully.")
print(f"Number of classes: {num_classes}")
print(f"Class names: {class_names}")

custom_s_voc.yaml file created successfully.
Number of classes: 4
Class names: ['trafficlight', 'stop', 'speedlimit', 'crosswalk']


In [19]:
results = model.train(
    data='custom_s_voc.yaml',
    epochs=20,
    batch=32,
    imgsz=640,
    device=0,
    workers=2,
    name='custom_s'
)
print(results)

Ultralytics 8.3.248 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=custom_s_voc.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=custom_s, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretra

In [20]:
# # 학습된 모델 로드 후 validation
# model = YOLO("/content/runs/detect/custom_s/weights/best.pt")

# # 검증 수행
# results = model.val(
#     data="custom_s_voc.yaml",  # 데이터셋 설정
#     imgsz=640,               # 이미지 크기
#     iou=0.5,                 # IoU 임계값
#     batch=32,                # 배치 크기
#     device=0,                # GPU 사용
#     workers=2,               # 데이터 로드 시 병렬 처리할 워커 수
#     half=True,               # FP16 연산 활성화 (속도 향상)
#     split="val"             # 테스트 데이터셋을 사용
# )
# print(results)

In [21]:
# VOC/custom에 test에 있는 이미지 랜덤하게 5장 뽑기
Path('VOC/custom').mkdir(parents=True, exist_ok=True)

test_images_dir = Path('VOC/test/images')
custom_dir = Path('VOC/custom')

all_test_image_files = sorted([p for p in test_images_dir.iterdir() if p.suffix == '.png'])

num_files_to_copy = min(5, len(all_test_image_files)) # Ensure we don't try to copy more files than exist
selected_files = random.sample(all_test_image_files, num_files_to_copy)

copied_file_names = []
for img_path in selected_files:
    shutil.copy2(img_path, custom_dir / img_path.name)
    copied_file_names.append(img_path.name)

print(f"Successfully copied {len(copied_file_names)} image files to VOC/custom.")
print("Copied files:")
for name in copied_file_names:
    print(f"- {name}")

Successfully copied 5 image files to VOC/custom.
Copied files:
- road346.png
- road673.png
- road255.png
- road393.png
- road176.png


In [22]:
# 사진 주고 분석 검증
model = YOLO("/content/runs/detect/custom_s/weights/best.pt")

# 객체 탐지 수행
results = model.predict(
    source="/content/VOC/custom",  # 테스트 이미지 폴더
    imgsz=640,           # 입력 이미지 크기
    conf=0.25,           # 신뢰도(Confidence) 임계값
    device=0,            # GPU 사용 (CPU 사용 시 "cpu")
    save=True,           # 탐지 결과 저장
    save_txt=True,       # 탐지 결과를 txt 형식으로 저장 (YOLO 포맷)
    save_conf=True       # 탐지된 객체의 신뢰도 점수도 저장
)
print(results)


image 1/5 /content/VOC/custom/road176.png: 640x480 1 trafficlight, 1 crosswalk, 52.1ms
image 2/5 /content/VOC/custom/road255.png: 640x480 1 speedlimit, 12.0ms
image 3/5 /content/VOC/custom/road346.png: 640x480 1 speedlimit, 12.0ms
image 4/5 /content/VOC/custom/road393.png: 640x480 1 speedlimit, 12.0ms
image 5/5 /content/VOC/custom/road673.png: 640x480 1 stop, 1 speedlimit, 12.0ms
Speed: 1.9ms preprocess, 20.0ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 480)
Results saved to /content/runs/detect/predict
5 labels saved to /content/runs/detect/predict/labels
[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'trafficlight', 1: 'stop', 2: 'speedlimit', 3: 'crosswalk'}
obb: None
orig_img: array([[[191, 183, 172],
        [197, 183, 174],
        [199, 184, 176],
        ...,
        [205, 186, 176],
        [203, 186, 175],
        [204, 187, 176]],

       [[192, 183, 172],
  

In [23]:
!jupyter nbconvert --to markdown "/content/drive/MyDrive/KDT/분석과제/YOLO.ipynb"

[NbConvertApp] Converting notebook /content/drive/MyDrive/KDT/분석과제/YOLO.ipynb to markdown
[NbConvertApp] Writing 99056 bytes to /content/drive/MyDrive/KDT/분석과제/YOLO.md
